# Module 2A - Data Wrangling and Data Aggregation

## Data Wrangling

### Hierarchical Indexing

Series Example

In [1]:
import numpy as np
import pandas as pd
data = pd.Series(np.random.randn(9), 
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'], [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -1.106485
   2   -0.799967
   3    1.223796
b  1    0.783966
   3    0.249823
c  1    0.925662
   2   -0.835584
d  2   -0.128676
   3    0.857291
dtype: float64

In [2]:
# Selection from the "outer level"
data['b']

1    0.783966
3    0.249823
dtype: float64

In [3]:
data['b':'c']

b  1    0.783966
   3    0.249823
c  1    0.925662
   2   -0.835584
dtype: float64

In [4]:
# Selection from the "inner level"
data[:,2]

a   -0.799967
c   -0.835584
d   -0.128676
dtype: float64

In [5]:
# Rearranging the data into a dataframe
data.unstack()

,1,2,3
a,-1.106485,-0.799967,1.223796
b,0.783966,NaN,0.249823
c,0.925662,-0.835584,NaN
d,NaN,-0.128676,0.857291


In [6]:
# Inverse of unstack is stack
data.unstack().stack()

a  1   -1.106485
   2   -0.799967
   3    1.223796
b  1    0.783966
   3    0.249823
c  1    0.925662
   2   -0.835584
d  2   -0.128676
   3    0.857291
dtype: float64

Dataframe examples

In [7]:
# Either axis can have a hierarchical level
frame = pd.DataFrame(np.arange(12).reshape((4, 3)), 
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]], 
                     columns=[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [8]:
# Hierarchical levels can have names:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [9]:
# Partial indexing to select groups of columns
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

#### Reordering and Sorting Levels

In [10]:
# Swaplevel takes two level numers or names and returns a new object with levels interchanged
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [11]:
# sort_index sorts data using only values on a single level:
frame.sort_index(level = 1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [12]:
# Often used after swapping levels
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

#### Summary Statistics by Level

In [13]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [14]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

#### Indexing with a DataFrame's Columns

In [15]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1), 
                      'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'], 
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [16]:
# set_index function creates new DataFrame using one or more of its columns as the index
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [17]:
# reset_index function does the opposite - converts index levels to columns
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## Combining and Merging Datasets

### Database-Style DataFrame Joins

In [18]:
df1 = pd.DataFrame({'key1': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})
df2 = pd.DataFrame({'key2': ['a', 'b', 'd'], 'data2': range(3)})
df1

,key1,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [19]:
df2

,key2,data2
0,a,0
1,b,1
2,d,2


In [20]:
# Inner Join
pd.merge(df1, df2, left_on = 'key1', right_on = 'key2')

,key1,data1,key2,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [21]:
# Outer join
pd.merge(df1, df2, left_on = 'key1', right_on = 'key2', how = 'outer')

,key1,data1,key2,data2
0,b,0.0,b,1.0
1,b,1.0,b,1.0
2,b,6.0,b,1.0
3,a,2.0,a,0.0
4,a,4.0,a,0.0
5,a,5.0,a,0.0
6,c,3.0,NaN,NaN
7,NaN,NaN,d,2.0


### Merging on Index

In [22]:
# Merging when merge key(s) in index
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'], 'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [23]:
right1

,group_val
a,3.5
b,7.0


In [24]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


## Reshaping and Pivoting

### Reshaping with Hierarchical Indexing

In [25]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)), 
                    index=pd.Index(['Ohio', 'Colorado'], name='state'), 
                    columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [26]:
# stack() pivots the columns into rows producing a series:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

Melt() method

In [27]:
sales_wide = pd.read_csv('Sales - Wide.csv')
sales_wide

,Store_Id,Year,Q1_Sales,Q2_Sales,Q3_Sales,Q4_Sales
0,A001,2018,"55,000,000","45,000,000","22,000,000","50,000,000"
1,A002,2018,"98,000,000","70,000,000","60,000,000","60,000,000"


In [28]:
sales_wide.melt(id_vars = ["Store_Id", "Year"], var_name = "Quarter", value_name = "Sales")

,Store_Id,Year,Quarter,Sales
0,A001,2018,Q1_Sales,"55,000,000"
1,A002,2018,Q1_Sales,"98,000,000"
2,A001,2018,Q2_Sales,"45,000,000"
3,A002,2018,Q2_Sales,"70,000,000"
4,A001,2018,Q3_Sales,"22,000,000"
5,A002,2018,Q3_Sales,"60,000,000"
6,A001,2018,Q4_Sales,"50,000,000"
7,A002,2018,Q4_Sales,"60,000,000"


Pivot() method

In [29]:
medical_data_long = pd.read_csv("Medical Data - Long.csv")
medical_data_long

,Name,Measurement,Value
0,Alice,Age,34
1,Alice,Gender,Female
2,Alice,Weight,115
3,Bob,Age,35
4,Bob,Weight,160
5,Bob,Gender,Male
6,Christine,Age,38
7,Christine,Gender,Female
8,Christine,Weight,125


In [30]:
medical_data_long.pivot(index = "Name", columns = "Measurement", values = "Value")

Measurement,Age,Gender,Weight
Name,,,
Alice,34,Female,115
Bob,35,Male,160
Christine,38,Female,125


# Data Aggregation

In [31]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'], 
                   'key2' : ['one', 'two', 'one', 'two', 'one'], 
                   'data1' : np.random.randn(5), 
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,0.623411,1.334739
1,a,two,-0.654141,-1.146278
2,b,one,-0.804319,1.626476
3,b,two,0.847847,-0.435078
4,a,one,0.070732,0.854882


Example:  compute the mean of the data1 column using labels from key1

In [32]:
#  Step 1:  create a GroupBy object
grouped = df['data1'].groupby(df['key1'])
grouped

In [33]:
grouped.mean()

key1
a    0.013334
b    0.021764
Name: data1, dtype: float64

In [34]:
# Grouping with two keys
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.347072
      two    -0.654141
b     one    -0.804319
      two     0.847847
Name: data1, dtype: float64

In [35]:
#  Result is a Series with a hierarchical index
type(means)

pandas.core.series.Series

In [36]:
# Grouping by information in the dataframe
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.013334,0.347781
b,0.021764,0.595699


In [37]:
df.groupby('key1').size()

key1
a    3
b    2
dtype: int64

In [38]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.347072  1.094811
     two  -0.654141 -1.146278
b    one  -0.804319  1.626476
     two   0.847847 -0.435078

In [39]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [40]:
#  Aggregating only selected columns
df['data1'].groupby(df['key1']).mean()

key1
a    0.013334
b    0.021764
Name: data1, dtype: float64

In [41]:
#  Equivalent syntax
df.groupby('key1')['data1'].mean()

key1
a    0.013334
b    0.021764
Name: data1, dtype: float64